In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
def load_train_data():
    train_file = 'train.json.zip'
    train = pd.read_json(train_file, orient='records', convert_dates=['created'])
    train.reset_index(drop=True, inplace=True)
    return train

def load_test_data():
    test_file = 'test.json.zip'
    test = pd.read_json(test_file, orient='records', convert_dates=['created'])
    test.reset_index(drop=True, inplace=True)
    return test


# Data Preparation

In [3]:
# load train data
train = load_train_data()

# remove outliers
# latitude
lower = np.percentile(train['latitude'].values, 1)
upper = np.percentile(train['latitude'].values, 99)
train = train[train['latitude'] > lower]
train = train[train['latitude'] < upper]
# train.latitude.hist()

# longitude
lower = np.percentile(train['longitude'].values, 1)
upper = np.percentile(train['longitude'].values, 99)
train = train[train['longitude'] > lower]
train = train[train['longitude'] < upper]
# train.longitude.hist()

# price
upper = np.percentile(train['price'].values, 99)
train = train[train['price'] < upper]
# train.price.hist()

train.reset_index(drop=True, inplace=True)

# Display all building on NYC map

In [4]:
# import descartes
# import geopandas as gpd
# from shapely.geometry import Point

# def get_geometry(data):
#     crs = {'init': 'epsg:4326'}
#     geometry=[Point(xy)for xy in zip(data['longitude'], data['latitude'])]
#     geo_df = gpd.GeoDataFrame(data,
#                             crs = crs,
#                             geometry = geometry)
#     geo_df.reset_index(inplace=True)
#     return geo_df


In [5]:
# #Load backgroud NYC map data
# vmap = gpd.read_file('NYC_street_map.shp')
# vmap_geo = vmap.to_crs("EPSG:4326")

# #Process the input data
# train_geo = get_geometry(train)

# fig, ax = plt.subplots(figsize = (15,15))
# vmap_geo.plot(ax=ax, alpha=0.4, color="grey", edgecolor = 'k')
# train_geo.plot(ax = ax, markersize = 10, color = 'red', marker = 'o', label='Building')
# plt.title('Building location distribution')
# plt.legend()
# plt.show()

In [6]:
# map_boundary = gpd.read_file('NYC_boundary.shp')
# map_boundary_geo = map_boundary.to_crs("EPSG:4326")

# fig, ax = plt.subplots(figsize = (15,15))
# map_boundary_geo.plot(ax=ax, alpha=0.4, color='black', edgecolor='k')
# train_geo.plot(ax = ax, markersize = 10, color = 'red', marker = 'o', label='Building')
# plt.title('Building location distribution')
# plt.legend()
# plt.show()

# Feature Selection

In [7]:
# Adding feature manager id
manager_id_df = train.groupby(['manager_id']).agg({'building_id': 'count'}).reset_index()
manager_number = manager_id_df['building_id'].shape[0]
manager_id_df['building_id'] = np.linspace(1, manager_number, num=manager_number, dtype=int)

train = train.merge(manager_id_df, how='left', on='manager_id')
del train['building_id_x']
train = train.rename(columns={'building_id_y': 'manager_number'})


In [8]:
# # finding all features
# features_dict = {}
# def get_features(features):
#     for f in features:
#         f = f.strip('!')
#         f = f.strip('*')
#         f = f.strip(' ')
        
#         if f in features_dict.keys():
#             features_dict[f] += 1
#         else:
#             features_dict[f] = 1
            
# temp = train['features'].apply(get_features)
# features_dict.keys()

# feature_columns = {}
# feature_list = []
# for key in features_dict.keys():
#     if features_dict[key] >= 20000:
#         feature_list.append(key)
#         feature_columns[key] = []
        
# def get_feature_columns(features):
#     for key in feature_columns:
#         if key in features:
#             feature_columns[key].append(True)
#         else:
#             feature_columns[key].append(False)
            
# temp = train['features'].apply(get_feature_columns)
# train = pd.concat([train, pd.DataFrame.from_dict(feature_columns)], axis=1)


# using external dataset
subway = pd.read_csv('NYC_Transit_Subway_Entrance_And_Exit_Data.csv')
subway = subway[['Station Name', 'Station Latitude', 'Station Longitude']]
subway = subway.groupby(['Station Name']).mean().reset_index(drop=True)

from math import cos, asin, sqrt
# https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula/21623206
def distance_pair(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295     #Pi/180
    a = 0.5 - cos((lat2 - lat1) * p)/2 +  cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
    d_2_point = 6371 *2 * asin(sqrt(a)) #2*R*asin...
    return d_2_point
distance_pairs = np.vectorize(distance_pair)

def get_nearby_subway(location):
    distances = distance_pairs(location[0], location[1], subway['Station Latitude'], subway['Station Longitude'])    
    return distances[distances < 1].shape[0]
    
def get_subway_distance(location):
    distances = distance_pairs(location[0], location[1], subway['Station Latitude'], subway['Station Longitude'])    
    return min(distances)

train['feature_number'] = train['features'].apply(len)
train['room_number'] = train['bedrooms'] + train['bathrooms']
train['photo_number'] = train['photos'].apply(len)
train['location'] = train[['latitude', 'longitude']].values.tolist()
train['nearby_subway'] = train['location'].apply(get_nearby_subway)
train['subway_distance'] = train['location'].apply(get_subway_distance)


# Features Selection Using Embedded Method (LassoCV())

In [9]:
# https://towardsdatascience.com/feature-selection-with-pandas-e3690ad8504b

In [10]:
# X_features = ['feature_number', 'room_number', 'manager_number', 'photo_number', 'nearby_subway', 'subway_distance', 'price']
# from sklearn.linear_model import LassoCV
# train['target'] = train['interest_level'].apply(lambda x: 0 if x=='low' else 1 if x=='medium' else 2)
# target = train['target']
# inputs = train[X_features]

# best_features = LassoCV().fit(inputs, target)
# print("Best alpha using built-in LassoCV: %f" % best_features.alpha_)
# print("Best score using built-in LassoCV: %f" %best_features.score(inputs, target))
# coef = pd.Series(best_features.coef_, index=inputs.columns)
# print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")
# imp_coef = coef.sort_values()
# import matplotlib
# matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
# imp_coef.plot(kind = "barh")
# plt.title("Feature importance using Lasso Model")

# Training Model

In [11]:
X_features = ['feature_number', 'room_number', 'photo_number', 'nearby_subway', 'subway_distance', 'price']

In [184]:
#Logistic Regression Model 1

X = train[X_features]
y = train['interest_level']
logistic_model = LogisticRegression(multi_class='multinomial', max_iter=800)

# scores = cross_val_score(logistic_model, X, y, cv=5)
# print("Accuracy: %f (+/- %f)" % (scores.mean(), scores.std() * 2))

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y)

# logistic_model.fit(X_train, y_train)
# score_train = logistic_model.score(X_train, y_train)
# score_test = logistic_model.score(X_test, y_test)
# print("Accuracy for training data: %f" % score_train.mean())
# print("Accuracy for testing data: %f" % score_test.mean())


Accuracy: 0.690504 (+/- 0.006536)


In [177]:
# Logistic Regression Model 2
train['high'] = train['interest_level'].apply(lambda x: True if x=='high' else False)
train['medium'] = train['interest_level'].apply(lambda x: True if x=='medium' else False)
train['low'] = train['interest_level'].apply(lambda x: True if x=='low' else False)

X = train[X_features]
y_high = train['high']
y_medium = train['medium']
y_low = train['low']

lr_high = LogisticRegression(multi_class='ovr')
lr_medium = LogisticRegression(multi_class='ovr')
lr_low = LogisticRegression(multi_class='ovr')

from sklearn.model_selection import cross_val_score
scores_high = cross_val_score(lr_high, X, y_high, cv=5)
print("Accuracy for lr_high: %f (+/- %f)" % (scores_high.mean(), scores_high.std() * 2))
scores_medium = cross_val_score(lr_medium, X, y_medium, cv=5)
print("Accuracy for lr_medium: %f (+/- %f)" % (scores_medium.mean(), scores_medium.std() * 2))
scores_low = cross_val_score(lr_low, X, y_low, cv=5)
print("Accuracy for lr_low: %f (+/- %f)" % (scores_low.mean(), scores_low.std() * 2))


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

Accuracy for lr_high: 0.927060 (+/- 0.000439)


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy for lr_medium: 0.764425 (+/- 0.003866)
Accuracy for lr_low: 0.704316 (+/- 0.006808)


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


# Testing Model

In [14]:
test = load_test_data()

In [15]:
test['feature_number'] = test['features'].apply(len)
test['room_number'] = test['bedrooms'] + test['bathrooms']
test['photo_number'] = test['photos'].apply(len)
test['location'] = test[['latitude', 'longitude']].values.tolist()
test['nearby_subway'] = test['location'].apply(get_nearby_subway)
test['subway_distance'] = test['location'].apply(get_subway_distance)
test = test.merge(manager_id_df, how='left', on='manager_id')
del test['building_id_x']
test = test.rename(columns={'building_id_y': 'manager_number'})

In [16]:
# test = pd.concat([test, pd.DataFrame.from_dict(feature_columns)], axis=1)

In [17]:
test_null = test[test['manager_number'].isnull()]
test_null['manager_number'] = np.zeros(test_null.shape[0])
test = test.dropna(subset=['manager_number'])
test = pd.concat([test, test_null])
test = test.astype({'manager_number':'int64'})
# test = test.astype({'Dogs Allowed':'bool', 'Hardwood Floors':'bool', 'Cats Allowed':'bool', 'Elevator':'bool'})

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
X_test = test[X_features]

In [56]:
# # Model 1
# logistic_model.fit(X, y)
# y_test = logistic_model.predict_proba(X_test)
# test_data = {'listing_id':test['listing_id'], 
#              'high':y_test[:,0], 
#             'medium' : y_test[:,2], 
#             'low' : y_test[:,1]}


In [20]:
# Model 2
lr_high.fit(X, y_high)
lr_medium.fit(X, y_medium)
lr_low.fit(X, y_low)

y_test_high = lr_high.predict_proba(X_test)[:,1]
y_test_medium = lr_medium.predict_proba(X_test)[:,1]
y_test_low = lr_low.predict_proba(X_test)[:,1]


test_data = {'listing_id' : test['listing_id'], 
             'high' : y_test_high, 
             'medium' : y_test_medium, 
             'low' : y_test_low}

In [21]:
data = pd.DataFrame(data=test_data)
data

,listing_id,high,medium,low
0,7142618,0.070164,0.261234,0.687964
1,7210040,0.145642,0.278017,0.537015
2,7174566,0.093342,0.200184,0.698756
3,7191391,0.131381,0.290551,0.573149
4,7171695,0.084836,0.275697,0.612100
...,...,...,...,...
74282,6946524,0.125432,0.284312,0.586533
74312,6859027,0.106815,0.279251,0.594247
74349,6824740,0.120766,0.262940,0.584180
74517,6875490,0.000198,0.025620,0.990615


In [22]:
# data.to_csv('submission.csv', index=False)

In [23]:
X_features

['feature_number',
 'room_number',
 'photo_number',
 'nearby_subway',
 'subway_distance',
 'price']

# ROC curve

In [175]:
from sklearn.metrics import roc_auc_score

In [181]:
# model 2
# lr_high
X_train, X_test, y_train, y_test = train_test_split(X, y_high)
y_score = lr_high.fit(X_train, y_train).predict(X_test)
print(roc_auc_score(y_test, y_score))

# lr_medium
X_train, X_test, y_train, y_test = train_test_split(X, y_medium)
y_score = lr_medium.fit(X_train, y_train).predict(X_test)
print(roc_auc_score(y_test, y_score))

# lr_low
X_train, X_test, y_train, y_test = train_test_split(X, y_low)
y_score = lr_low.fit(X_train, y_train).predict(X_test)
print(roc_auc_score(y_test, y_score))


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.501000716056738
0.5045420420420421
0.5601835486289282


# F1_score

In [187]:
# # model 1
# from sklearn.metrics import f1_score
# y = y.apply(lambda x: 0 if x=='low' else 1 if x=='medium' else 2)
# X_train, X_test, y_train, y_test = train_test_split(X, y)
# y_score = logistic_model.fit(X_train, y_train).predict(X_test)
# f1_score(y_test, y_score, average='micro')

0.686503538238554